# 05 — Modules 5–6: Universe filtering & Ranking

Universe building merges the indicator outputs and applies filters:

Examples of filters:
- price between min and max
- ATR% below threshold
- trend_ok must be True
- optionally RS must be positive

Ranking then scores the remaining tickers using percentile ranks of:
- momentum
- relative strength

Output:
- a ranked candidate list with `score` and `rank`

In [1]:
# If running from repo root and editable install is not done:
# pip install -e ".[dev]"

import pandas as pd
pd.set_option("display.width", 140)
pd.set_option("display.max_columns", 50)

> Tip: many modules assume the benchmark **SPY** is present (for Relative Strength).
> When using real tickers, include SPY:
>
> `tickers = ["AAPL","MSFT","NVDA","SPY"]`

In [2]:
from swing_screener.data.market_data import fetch_ohlcv, MarketDataConfig
from swing_screener.screeners.universe import UniverseConfig, UniverseFilterConfig, eligible_universe
from swing_screener.screeners.ranking import RankingConfig, top_candidates

tickers = ["AAPL","MSFT","NVDA","AMZN","META","GOOGL","TSLA","SPY"]
ohlcv = fetch_ohlcv(tickers, MarketDataConfig(start="2022-01-01"))

ucfg = UniverseConfig(
    filt=UniverseFilterConfig(
        min_price=10,
        max_price=2000,
        max_atr_pct=10.0,
        require_trend_ok=True,
        require_rs_positive=False,
    )
)

univ = eligible_universe(ohlcv, ucfg)
univ.head()

,last,sma20,sma50,sma200,trend_ok,dist_sma50_pct,dist_sma200_pct,atr14,atr_pct,mom_6m,mom_12m,rs_6m,is_eligible,reason
GOOGL,335.970001,315.156001,306.140384,223.444596,True,9.743771,50.359421,6.583570,1.959571,0.848849,0.756331,0.727156,True,ok
TSLA,447.200012,461.338503,444.147202,366.503400,True,0.687342,22.017971,14.218569,3.179465,0.438960,0.132898,0.317268,True,ok
AAPL,261.049988,268.816496,272.193662,233.188190,True,-4.094024,11.948203,3.890707,1.490407,0.251014,0.107159,0.129322,True,ok
NVDA,185.809998,184.417500,185.585796,162.900744,True,0.120807,14.063321,4.704992,2.532152,0.088640,0.367529,-0.033053,True,ok
AMZN,242.600006,233.308000,234.039400,218.529150,True,3.657763,11.014940,5.051430,2.082205,0.071791,0.108066,-0.049901,True,ok


In [3]:
ranked = top_candidates(univ, RankingConfig(top_n=10))
ranked[["rank","score","last","atr_pct","mom_6m","rs_6m","trend_ok"]]

,rank,score,last,atr_pct,mom_6m,rs_6m,trend_ok
GOOGL,1,1.00,335.970001,1.959571,0.848849,0.727156,True
TSLA,2,0.73,447.200012,3.179465,0.438960,0.317268,True
NVDA,3,0.54,185.809998,2.532152,0.088640,-0.033053,True
AAPL,4,0.46,261.049988,1.490407,0.251014,0.129322,True
AMZN,5,0.27,242.600006,2.082205,0.071791,-0.049901,True


If you get an empty universe, loosen filters (e.g. `require_trend_ok=False` or `max_atr_pct=15`).